# Deploy MCP Server with Amazon Bedrock AgentCore Example

## Overview

This tutorial demonstrates how to use the Amazon Bedrock AgentCore Python SDK to deploy existing MCP server tools to AgentCore Runtime. We will use the MCP server tools from `server/app.py` in the project, including:

- `search_website`: Website search tool
- `count_s3_buckets`: S3 bucket counting tool
- `generate_image_with_context`: Image generation tool
- `get_comfyui_config`: ComfyUI configuration retrieval tool
- `generate_video_with_context`: Video generation tool

### Tutorial Details

| Information | Details |
|:--------------------|:----------------------------------------------------------|
| Tutorial Type | Tool Hosting |
| Tool Type | MCP server |
| Tutorial Components | Hosting MCP server on AgentCore Runtime |
| Example Complexity | Intermediate |
| SDKs Used | Amazon BedrockAgentCore Python SDK and MCP |

### Key Features

* Use existing MCP server tools
* Configure OAuth authentication (Cognito)
* Deploy to Amazon Bedrock AgentCore Runtime
* Test deployed MCP server directly


## Prerequisites

To execute this tutorial, you need:
* Python 3.10+
* Configured AWS credentials
* Amazon Bedrock AgentCore SDK
* MCP (Model Context Protocol) library
* Docker runtime environment
* SerpAPI API Key (for website search functionality, optional)

In [ ]:
# Install necessary dependencies
%pip install mcp bedrock-agentcore-starter-toolkit boto3 requests

## Create Adapted MCP Server

We need to create an MCP server adapted for AgentCore Runtime that uses FastMCP and supports stateless HTTP.
This MCP server contains all the tool functionality from the original `server/app.py`.

In [ ]:
%%writefile agentcore_mcp_server.py
from mcp.server.fastmcp import FastMCP
import boto3
import os
import requests
from typing import Dict
import base64
import json
import time
import uuid

# 创建 FastMCP 实例，配置为 stateless HTTP
mcp = FastMCP(host="0.0.0.0", stateless_http=True)

# 从环境变量获取配置
SERPAPI_API_KEY = os.environ.get('SERPAPI_API_KEY', 'your_serpapi_key_here')
COMFYUI_SERVER_URL = os.environ.get('COMFYUI_SERVER_URL', 'http://localhost:8188')
COMFYUI_TIMEOUT = int(os.environ.get('COMFYUI_TIMEOUT', '300'))
COMFYUI_POLL_INTERVAL = int(os.environ.get('COMFYUI_POLL_INTERVAL', '2'))
COMFYUI_MAX_RETRIES = int(os.environ.get('COMFYUI_MAX_RETRIES', '3'))

# ComfyUI helper functions
def generate_seed() -> int:
    """Generate random seed"""
    return int(time.time() * 1000) % 2147483647

def validate_image_data(image_data: str) -> tuple[str, str]:
    """Validate and extract image data"""
    if not image_data.startswith('data:image/'):
        raise ValueError("Invalid image format. Expected data URL format (data:image/...)")
    
    try:
        header, data = image_data.split(',', 1)
        mime_type = header.split(';')[0].split(':')[1]
        # Validate base64 data
        base64.b64decode(data)
        return mime_type, data
    except Exception as e:
        raise ValueError(f"Failed to decode image: {str(e)}")

def test_comfyui_connectivity() -> bool:
    """Test ComfyUI server connectivity"""
    try:
        response = requests.get(f"{COMFYUI_SERVER_URL}/queue", timeout=5)
        return response.status_code == 200
    except:
        return False

def create_flux_t2i_workflow(prompt: str, width: int, height: int, steps: int, cfg_scale: float, seed: int) -> Dict:
    """Create FLUX text-to-image workflow (based on original project's flux_t2i.json)"""
    return {
        "6": {
            "inputs": {
                "text": prompt,
                "speak_and_recognation": {
                    "__value__": [False, True]
                },
                "clip": ["41", 0]
            },
            "class_type": "CLIPTextEncode",
            "_meta": {
                "title": "CLIP Text Encode (Positive Prompt)"
            }
        },
        "8": {
            "inputs": {
                "samples": ["31", 0],
                "vae": ["40", 0]
            },
            "class_type": "VAEDecode",
            "_meta": {
                "title": "VAE解码"
            }
        },
        "9": {
            "inputs": {
                "filename_prefix": "ComfyUI",
                "images": ["8", 0]
            },
            "class_type": "SaveImage",
            "_meta": {
                "title": "保存图像"
            }
        },
        "31": {
            "inputs": {
                "seed": seed,
                "steps": steps,
                "cfg": cfg_scale,
                "sampler_name": "euler",
                "scheduler": "simple",
                "denoise": 1,
                "model": ["38", 0],
                "positive": ["6", 0],
                "negative": ["33", 0],
                "latent_image": ["42", 0]
            },
            "class_type": "KSampler",
            "_meta": {
                "title": "K采样器"
            }
        },
        "33": {
            "inputs": {
                "text": "bad quality, blurry, low resolution",
                "speak_and_recognation": {
                    "__value__": [False, True]
                },
                "clip": ["41", 0]
            },
            "class_type": "CLIPTextEncode",
            "_meta": {
                "title": "CLIP Text Encode (Negative Prompt)"
            }
        },
        "38": {
            "inputs": {
                "unet_name": "flux1-dev-fp8.safetensors",
                "weight_dtype": "fp8_e4m3fn"
            },
            "class_type": "UNETLoader",
            "_meta": {
                "title": "UNet加载器"
            }
        },
        "40": {
            "inputs": {
                "vae_name": "FLUX1/ae.safetensors"
            },
            "class_type": "VAELoader",
            "_meta": {
                "title": "加载VAE"
            }
        },
        "41": {
            "inputs": {
                "clip_name1": "clip_l.safetensors",
                "clip_name2": "t5xxl_fp8_e4m3fn.safetensors",
                "type": "flux",
                "device": "default"
            },
            "class_type": "DualCLIPLoader",
            "_meta": {
                "title": "双CLIP加载器"
            }
        },
        "42": {
            "inputs": {
                "width": width,
                "height": height,
                "batch_size": 1
            },
            "class_type": "EmptyLatentImage",
            "_meta": {
                "title": "空Latent图像"
            }
        }
    }

def create_wanvideo_t2v_workflow(prompt: str, steps: int, cfg_scale: float, seed: int, frame_rate: int) -> Dict:
    """创建 WanVideo 文本到视频工作流（基于原始项目的 wanv_t2v.json）"""
    return {
        "11": {
            "inputs": {
                "t5_model_name": "umt5-xxl-enc-bf16.safetensors",
                "dtype": "bf16",
                "device": "offload_device"
            },
            "class_type": "LoadWanVideoT5TextEncoder",
            "_meta": {
                "title": "Load WanVideo T5 Text Encoder"
            }
        },
        "16": {
            "inputs": {
                "positive_prompt": prompt,
                "negative_prompt": "bad quality video, blurry, low resolution, static",
                "enable_positive_prompt": True,
                "t5": ["11", 0]
            },
            "class_type": "WanVideoTextEncode",
            "_meta": {
                "title": "WanVideo Text Encode"
            }
        },
        "22": {
            "inputs": {
                "model_name": "WanVideo/Wan2_1-T2V-14B_fp8_e4m3fn.safetensors",
                "dtype": "bf16",
                "weight_dtype": "fp8_e4m3fn",
                "device": "offload_device",
                "attention_mode": "sdpa",
                "block_swap_args": ["39", 0]
            },
            "class_type": "WanVideoModelLoader",
            "_meta": {
                "title": "WanVideo Model Loader"
            }
        },
        "27": {
            "inputs": {
                "steps": steps,
                "cfg": cfg_scale,
                "denoise": 5,
                "seed": seed,
                "seed_mode": "fixed",
                "enable_vae_tiling": True,
                "sampler_name": "dpm++",
                "scheduler": 0,
                "model": ["22", 0],
                "text_embeds": ["16", 0],
                "image_embeds": ["37", 0]
            },
            "class_type": "WanVideoSampler",
            "_meta": {
                "title": "WanVideo Sampler"
            }
        },
        "28": {
            "inputs": {
                "enable_vae_tiling": True,
                "tile_sample_min_height": 272,
                "tile_sample_min_width": 272,
                "tile_overlap_factor_height": 144,
                "tile_overlap_factor_width": 128,
                "vae": ["38", 0],
                "samples": ["27", 0]
            },
            "class_type": "WanVideoDecode",
            "_meta": {
                "title": "WanVideo Decode"
            }
        },
        "30": {
            "inputs": {
                "frame_rate": frame_rate,
                "loop_count": 0,
                "filename_prefix": "WanVideo_T2V",
                "format": "video/h264-mp4",
                "pix_fmt": "yuv420p",
                "crf": 19,
                "save_metadata": True,
                "trim_to_audio": False,
                "pingpong": False,
                "save_output": True,
                "images": ["28", 0]
            },
            "class_type": "VHS_VideoCombine",
            "_meta": {
                "title": "Video Combine"
            }
        },
        "37": {
            "inputs": {
                "width": 832,
                "height": 480,
                "frames": 81
            },
            "class_type": "WanVideoEmptyEmbeds",
            "_meta": {
                "title": "WanVideo Empty Embeds"
            }
        },
        "38": {
            "inputs": {
                "vae_name": "wanvideo/Wan2_1_VAE_bf16.safetensors",
                "dtype": "bf16"
            },
            "class_type": "WanVideoVAELoader",
            "_meta": {
                "title": "WanVideo VAE Loader"
            }
        },
        "39": {
            "inputs": {
                "block_swap_memory_threshold": 20
            },
            "class_type": "WanVideoBlockSwap",
            "_meta": {
                "title": "WanVideo Block Swap"
            }
        }
    }

def submit_comfyui_workflow(workflow: Dict) -> Dict:
    """提交工作流到 ComfyUI 并等待结果"""
    if not test_comfyui_connectivity():
        return {"error": "ComfyUI server is not accessible"}

    try:
        # 生成唯一的 prompt ID
        prompt_id = str(uuid.uuid4())

        # 提交工作流
        submit_url = f"{COMFYUI_SERVER_URL}/prompt"
        submit_payload = {
            "prompt": workflow,
            "client_id": prompt_id
        }

        response = requests.post(submit_url, json=submit_payload, timeout=30)
        if response.status_code != 200:
            return {"error": f"Failed to submit workflow (HTTP {response.status_code}): {response.text}"}

        submit_result = response.json()
        if not submit_result.get("prompt_id"):
            return {"error": "No prompt_id returned from ComfyUI"}

        actual_prompt_id = submit_result["prompt_id"]

        # 轮询完成状态
        return poll_comfyui_completion(actual_prompt_id)

    except Exception as e:
        return {"error": f"ComfyUI workflow submission failed: {str(e)}"}

def poll_comfyui_completion(prompt_id: str) -> Dict:
    """轮询 ComfyUI 工作流完成状态"""
    start_time = time.time()

    while time.time() - start_time < COMFYUI_TIMEOUT:
        try:
            # 检查队列状态
            queue_url = f"{COMFYUI_SERVER_URL}/queue"
            queue_response = requests.get(queue_url, timeout=30)

            if queue_response.status_code != 200:
                time.sleep(COMFYUI_POLL_INTERVAL)
                continue

            queue_data = queue_response.json()
            running = queue_data.get("queue_running", [])
            pending = queue_data.get("queue_pending", [])

            prompt_in_queue = any(item[1] == prompt_id for item in running + pending)

            if not prompt_in_queue:
                # 任务完成，获取结果
                return get_comfyui_result(prompt_id, start_time)

            time.sleep(COMFYUI_POLL_INTERVAL)

        except Exception as e:
            time.sleep(COMFYUI_POLL_INTERVAL)

    return {"error": f"ComfyUI generation timeout after {COMFYUI_TIMEOUT} seconds"}

def get_comfyui_result(prompt_id: str, start_time: float) -> Dict:
    """获取 ComfyUI 工作流结果"""
    try:
        # 获取历史记录
        history_url = f"{COMFYUI_SERVER_URL}/history/{prompt_id}"
        history_response = requests.get(history_url, timeout=30)

        if history_response.status_code != 200:
            return {"error": f"Failed to get ComfyUI history (HTTP {history_response.status_code})"}

        history_data = history_response.json()

        if prompt_id not in history_data:
            return {"error": "Prompt not found in history"}

        prompt_data = history_data[prompt_id]
        if "status" in prompt_data and prompt_data["status"].get("status_str") == "error":
            error_details = prompt_data["status"].get("messages", [])
            return {"error": f"ComfyUI execution error: {error_details}"}

        outputs = prompt_data.get("outputs", {})

        # 查找图像或视频输出
        for node_id, node_output in outputs.items():
            # 处理图像输出
            if "images" in node_output:
                images = node_output["images"]
                if images:
                    image_info = images[0]
                    if image_info.get('filename'):
                        image_url = f"{COMFYUI_SERVER_URL}/view?filename={image_info['filename']}&subfolder={image_info.get('subfolder', '')}&type={image_info.get('type', 'output')}"

                        image_response = requests.get(image_url, timeout=30)
                        if image_response.status_code == 200 and len(image_response.content) > 0:
                            image_base64 = base64.b64encode(image_response.content).decode('utf-8')
                            return {
                                "image_data": image_base64,
                                "generation_time": time.time() - start_time,
                                "prompt_id": prompt_id,
                                "image_size": len(image_response.content)
                            }

            # 处理视频输出
            elif "gifs" in node_output:
                videos = node_output["gifs"]
                if videos:
                    video_info = videos[0]
                    if video_info.get('filename'):
                        video_url = f"{COMFYUI_SERVER_URL}/view?filename={video_info['filename']}&subfolder={video_info.get('subfolder', '')}&type={video_info.get('type', 'output')}"

                        video_response = requests.get(video_url, timeout=60)
                        if video_response.status_code == 200 and len(video_response.content) > 0:
                            video_base64 = base64.b64encode(video_response.content).decode('utf-8')
                            return {
                                "video_data": video_base64,
                                "generation_time": time.time() - start_time,
                                "prompt_id": prompt_id,
                                "video_size": len(video_response.content),
                                "filename": video_info['filename']
                            }

        return {"error": "No valid images or videos found in ComfyUI output"}

    except Exception as e:
        return {"error": f"Failed to get workflow result: {str(e)}"}



@mcp.tool()
def search_website(search_term: str) -> Dict:
    """查询网站获取最新信息
    Args:
        search_term: 用户查询文本

    Returns:
        搜索结果字典
    """
    params = {
        "api_key": SERPAPI_API_KEY,
        "engine": "google",
        "q": search_term,
        "google_domain": "google.com",
        "gl": "us",
        "hl": "en"
    }

    url = "https://serpapi.com/search"
    try:
        response = requests.get(url, params=params, timeout=30)
        if response.status_code == 200:
            results = response.json()
            organic_results = results.get('organic_results', [])
            return {"search_result": organic_results}
        else:
            return {"search_result": f"Error: {response.status_code} - {response.text}"}
    except Exception as e:
        return {"search_result": f"Error: {str(e)}"}

@mcp.tool()
def count_s3_buckets() -> int:
    """计算 S3 存储桶的数量"""
    try:
        s3 = boto3.client('s3')
        response = s3.list_buckets()
        return len(response['Buckets'])
    except Exception as e:
        return f"Error counting S3 buckets: {str(e)}"

@mcp.tool()
def generate_image_with_context(
    prompt: str,
    context_image_base64: str = None,
    workflow_type: str = "text_to_image",
    width: int = 1024,
    height: int = 1024,
    steps: int = 20,
    cfg_scale: float = 7.0,
    seed: int = -1
) -> Dict:
    """使用 ComfyUI 生成图像，支持可选的上下文图像

    Args:
        prompt: 图像生成的文本描述
        context_image_base64: Base64 编码的上下文图像（可选）
        workflow_type: 工作流类型 (text_to_image, image_to_image, inpainting)
        width: 图像宽度（默认：1024）
        height: 图像高度（默认：1024）
        steps: 采样步数（默认：20）
        cfg_scale: CFG 引导比例（默认：7.0）
        seed: 随机种子（-1 为随机，默认：-1）

    Returns:
        生成的图像数据和元数据
    """
    try:
        # 验证输入
        if not prompt or not prompt.strip():
            return {"error": "Prompt cannot be empty"}

        if width <= 0 or height <= 0:
            return {"error": "Width and height must be positive"}

        if seed == -1:
            seed = generate_seed()

        # 检查 ComfyUI 连接
        if not test_comfyui_connectivity():
            return {"error": "ComfyUI server is not accessible"}

        # 使用与原始项目相同的 FLUX 文本到图像工作流
        workflow = create_flux_t2i_workflow(prompt, width, height, steps, cfg_scale, seed)

        # 提交工作流
        result = submit_comfyui_workflow(workflow)

        if result.get('error'):
            return {"error": result['error']}

        return {
            'type': 'image',
            'data': f"data:image/png;base64,{result['image_data']}",
            'mimeType': 'image/png',
            'metadata': {
                'prompt': prompt,
                'workflow_type': workflow_type,
                'dimensions': f"{width}x{height}",
                'steps': steps,
                'cfg_scale': cfg_scale,
                'seed': seed,
                'generation_time': result.get('generation_time', 0),
                'image_size': result.get('image_size', 0),
                'server_url': COMFYUI_SERVER_URL
            }
        }

    except Exception as e:
        return {"error": f"Image generation failed: {str(e)}"}

@mcp.tool()
def get_comfyui_config() -> Dict:
    """获取 ComfyUI 配置和可用工作流

    Returns:
        配置信息，包括可用工作流和预设
    """
    return {
        'server_url': COMFYUI_SERVER_URL,
        'available_workflows': ['text_to_image', 'image_to_image', 'text_to_video', 'image_to_video'],
        'workflow_presets': {
            'fast': {'steps': 15, 'cfg_scale': 6.0},
            'balanced': {'steps': 20, 'cfg_scale': 7.0},
            'quality': {'steps': 30, 'cfg_scale': 8.0}
        },
        'optimal_dimensions': {
            'square': (1024, 1024),
            'portrait': (768, 1024),
            'landscape': (1024, 768),
            'wide': (1152, 768),
            'tall': (768, 1152)
        },
        'config': {
            'timeout': COMFYUI_TIMEOUT,
            'poll_interval': COMFYUI_POLL_INTERVAL,
            'max_retries': COMFYUI_MAX_RETRIES
        },
        'connectivity': {
            'server_accessible': test_comfyui_connectivity(),
            'last_check': time.time()
        }
    }

@mcp.tool()
def generate_video_with_context(
    prompt: str,
    context_image_base64: str = None,
    workflow_type: str = "text_to_video",
    steps: int = 15,
    cfg_scale: float = 6.0,
    seed: int = -1,
    frame_rate: int = 16
) -> Dict:
    """使用 ComfyUI 生成视频，支持可选的上下文图像

    Args:
        prompt: 视频生成的文本描述
        context_image_base64: Base64 编码的上下文图像（image_to_video 需要）
        workflow_type: 工作流类型 (text_to_video, image_to_video)
        steps: 采样步数（默认：15）
        cfg_scale: CFG 引导比例（默认：6.0）
        seed: 随机种子（-1 为随机，默认：-1）
        frame_rate: 视频帧率（默认：16）

    Returns:
        生成的视频数据和元数据
    """
    try:
        # 验证输入
        if not prompt or not prompt.strip():
            return {"error": "Prompt cannot be empty"}

        if workflow_type == "image_to_video" and not context_image_base64:
            return {"error": "image_to_video 工作流需要 context_image_base64 参数"}

        if seed == -1:
            seed = generate_seed()

        # 检查 ComfyUI 连接
        if not test_comfyui_connectivity():
            return {"error": "ComfyUI server is not accessible"}

        # 使用与原始项目相同的 WanVideo 文本到视频工作流
        workflow = create_wanvideo_t2v_workflow(prompt, steps, cfg_scale, seed, frame_rate)

        # 如果有上下文图像，添加图像输入节点
        if context_image_base64 and workflow_type == "image_to_video":
            try:
                _, base64_data = validate_image_data(context_image_base64)
                workflow["6"] = {
                    "inputs": {
                        "image": base64_data
                    },
                    "class_type": "LoadImageBase64"
                }
                # 修改采样器以使用图像输入
                workflow["3"]["inputs"]["image"] = ["6", 0]
            except Exception as e:
                return {"error": f"Invalid context image: {str(e)}"}

        # 提交工作流
        result = submit_comfyui_workflow(workflow)

        if result.get('error'):
            return {"error": result['error']}

        # 检查是否获得了视频数据
        if 'video_data' in result:
            return {
                'type': 'video',
                'data': f"data:video/mp4;base64,{result['video_data']}",
                'mimeType': 'video/mp4',
                'metadata': {
                    'prompt': prompt,
                    'workflow_type': workflow_type,
                    'steps': steps,
                    'cfg_scale': cfg_scale,
                    'seed': seed,
                    'frame_rate': frame_rate,
                    'generation_time': result.get('generation_time', 0),
                    'video_size': result.get('video_size', 0),
                    'filename': result.get('filename', ''),
                    'server_url': COMFYUI_SERVER_URL
                }
            }
        else:
            return {"error": "No video data received from ComfyUI"}

    except Exception as e:
        return {"error": f"Video generation failed: {str(e)}"}

if __name__ == "__main__":
    mcp.run(transport="streamable-http")


## Create Dependencies File

Create requirements.txt file containing all necessary dependencies:

In [ ]:
%%writefile requirements.txt
mcp
boto3
requests
fastapi
uvicorn
starlette

## Setup Amazon Cognito Authentication

AgentCore Runtime requires OAuth authentication. We will use Amazon Cognito to provide JWT tokens for accessing the deployed MCP server.

In [ ]:
import boto3
import json
import time
from boto3.session import Session

def setup_cognito_user_pool():
    """Setup Cognito user pool and client"""
    cognito_client = boto3.client('cognito-idp')
    
    try:
        # Create user pool
        print("Creating Cognito user pool...")
        user_pool_response = cognito_client.create_user_pool(
            PoolName='MCPServerUserPool',
            Policies={
                'PasswordPolicy': {
                    'MinimumLength': 8,
                    'RequireUppercase': False,
                    'RequireLowercase': False,
                    'RequireNumbers': False,
                    'RequireSymbols': False
                }
            }
        )
        
        user_pool_id = user_pool_response['UserPool']['Id']
        print(f"✓ User pool created successfully: {user_pool_id}")
        
        # Create app client
        print("Creating app client...")
        client_response = cognito_client.create_user_pool_client(
            UserPoolId=user_pool_id,
            ClientName='MCPServerClient',
            GenerateSecret=False,
            ExplicitAuthFlows=[
                'ALLOW_USER_PASSWORD_AUTH',
                'ALLOW_REFRESH_TOKEN_AUTH'
            ]
        )
        
        client_id = client_response['UserPoolClient']['ClientId']
        print(f"✓ App client created successfully: {client_id}")
        
        # Create test user
        print("Creating test user...")
        cognito_client.admin_create_user(
            UserPoolId=user_pool_id,
            Username='testuser',
            TemporaryPassword='TempPass123!',
            MessageAction='SUPPRESS'
        )
        
        # Set permanent password
        cognito_client.admin_set_user_password(
            UserPoolId=user_pool_id,
            Username='testuser',
            Password='MyPassword123!',
            Permanent=True
        )
        print("✓ Test user created successfully")
        
        # Get access token
        print("Getting access token...")
        auth_response = cognito_client.initiate_auth(
            ClientId=client_id,
            AuthFlow='USER_PASSWORD_AUTH',
            AuthParameters={
                'USERNAME': 'testuser',
                'PASSWORD': 'MyPassword123!'
            }
        )
        
        access_token = auth_response['AuthenticationResult']['AccessToken']
        print("✓ 访问令牌获取成功")
        
        # 获取区域
        session = Session()
        region = session.region_name or 'us-east-1'
        
        discovery_url = f"https://cognito-idp.{region}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"
        
        return {
            'user_pool_id': user_pool_id,
            'client_id': client_id,
            'discovery_url': discovery_url,
            'bearer_token': access_token,
            'region': region
        }
        
    except Exception as e:
        print(f"❌ Cognito setup failed: {e}")
        raise

def create_agentcore_role(agent_name: str):
    """Create AgentCore IAM execution role"""
    iam_client = boto3.client('iam')
    
    role_name = f"AgentCore-{agent_name}-ExecutionRole"
    
    # Trust policy
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock-agentcore.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    try:
        # Create role
        role_response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description=f"Execution role for AgentCore {agent_name}"
        )
        
        # 附加基本执行策略
        policy_document = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "logs:CreateLogGroup",
                        "logs:CreateLogStream",
                        "logs:PutLogEvents",
                        "s3:ListAllMyBuckets",
                        "s3:GetBucketLocation"
                    ],
                    "Resource": "*"
                }
            ]
        }
        
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=f"{agent_name}-ExecutionPolicy",
            PolicyDocument=json.dumps(policy_document)
        )
        
        print(f"✓ IAM role created successfully: {role_name}")
        return role_response
        
    except Exception as e:
        print(f"❌ IAM role creation failed: {e}")
        raise

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("\n✅ Cognito setup completed")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")
print(f"Discovery URL: {cognito_config.get('discovery_url', 'N/A')}")

## Create IAM Execution Role

Create the necessary IAM role for AgentCore Runtime:

In [ ]:
tool_name = "mcp_server_sample"
print(f"Creating IAM role for {tool_name}...")
agentcore_iam_role = create_agentcore_role(agent_name=tool_name)
print(f"\n✅ IAM role creation completed")
print(f"Role ARN: {agentcore_iam_role['Role']['Arn']}")

## Configure AgentCore Runtime Deployment

Configure deployment using AgentCore SDK, including entry point, execution role, and authentication configuration:

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import os

# 获取当前区域
boto_session = Session()
region = boto_session.region_name or 'us-east-1'
print(f"使用 AWS 区域: {region}")

# 检查必需文件
required_files = ['agentcore_mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("✓ All required files found")

# Create AgentCore Runtime instance
agentcore_runtime = Runtime()

# Configure authentication
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

# Configure environment variables
environment_variables = {
    'SERPAPI_API_KEY': os.environ.get('SERPAPI_API_KEY', 'your_serpapi_key_here'),
    'COMFYUI_SERVER_URL': os.environ.get('COMFYUI_SERVER_URL', 'http://localhost:8188'),
    'COMFYUI_TIMEOUT': os.environ.get('COMFYUI_TIMEOUT', '300'),
    'COMFYUI_POLL_INTERVAL': os.environ.get('COMFYUI_POLL_INTERVAL', '2'),
    'COMFYUI_MAX_RETRIES': os.environ.get('COMFYUI_MAX_RETRIES', '3')
}

print("Configuring AgentCore Runtime...")
print(f"Environment variables configuration: {environment_variables}")
print("Note: Environment variables will be retrieved via os.environ in MCP server code")
response = agentcore_runtime.configure(
    entrypoint="agentcore_mcp_server.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=tool_name,
    # 注意：AgentCore 目前只支持 PUBLIC 网络模式
    # VPC 配置暂不支持，需要确保 ComfyUI 服务器可通过公网访问
    network_configuration={"networkMode": "PUBLIC"}
)
print("✅ Configuration completed")

## Launch MCP Server to AgentCore Runtime

Now deploy the MCP server to AgentCore Runtime:

In [ ]:
print("启动 MCP server 到 AgentCore Runtime...")
print("这可能需要几分钟时间...")
launch_result = agentcore_runtime.launch()
print("✅ 启动完成")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

## Check AgentCore Runtime Status

Check deployment status and wait for readiness:

In [ ]:
import time

print("检查 AgentCore Runtime 状态...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"初始状态: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"状态: {status} - 等待中...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✅ AgentCore Runtime 已就绪！")
else:
    print(f"⚠ AgentCore Runtime 状态: {status}")
    
print(f"最终状态: {status}")

## Store Configuration Information

Store Agent ARN and Cognito configuration to AWS Systems Manager and Secrets Manager:

In [ ]:
ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    # 存储 Cognito 凭证到 Secrets Manager
    try:
        cognito_credentials_response = secrets_client.create_secret(
            Name='mcp_server_sample/cognito/credentials',
            Description='MCP server 的 Cognito 凭证',
            SecretString=json.dumps(cognito_config)
        )
        print("✓ Cognito 凭证已存储到 Secrets Manager")
    except secrets_client.exceptions.ResourceExistsException:
        secrets_client.update_secret(
            SecretId='mcp_server_sample/cognito/credentials',
            SecretString=json.dumps(cognito_config)
        )
        print("✓ Cognito 凭证已更新到 Secrets Manager")

    # 存储 Agent ARN 到 Parameter Store
    agent_arn_response = ssm_client.put_parameter(
        Name='/mcp_server_sample/runtime/agent_arn',
        Value=launch_result.agent_arn,
        Type='String',
        Description='MCP server 的 Agent ARN',
        Overwrite=True
    )
    print("✓ Agent ARN 已存储到 Parameter Store")

    print("\n✅ 配置信息存储成功！")
    print(f"Agent ARN: {launch_result.agent_arn}")
    
except Exception as e:
    print(f"❌ 存储配置时出错: {e}")

## Create Remote Test Client

Create a client to test the deployed MCP server:

In [ ]:
# 直接在 notebook 中测试已部署的 MCP server
import asyncio
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def test_deployed_mcp_server():
    """测试已部署的 MCP server"""
    try:
        # 从存储的配置获取信息
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server_sample/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server_sample/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ 获取到认证信息")
        
        # 构建 MCP URL
        encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
        mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
        headers = {
            "authorization": f"Bearer {bearer_token}",
            "Content-Type": "application/json"
        }
        
        print(f"\n连接到 AgentCore Runtime...")
        
        async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("🔄 初始化 MCP 会话...")
                await session.initialize()
                print("✓ MCP 会话初始化成功")
                
                print("\n📋 获取可用工具列表...")
                tool_result = await session.list_tools()
                
                print("\n可用的 MCP 工具:")
                print("=" * 40)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print(f"\n✅ 成功！找到 {len(tool_result.tools)} 个工具")
                
                # 测试一个简单的工具
                print("\n🧪 测试 count_s3_buckets 工具...")
                try:
                    result = await session.call_tool(
                        name="count_s3_buckets",
                        arguments={}
                    )
                    print(f"S3 存储桶数量: {result.content[0].text}")
                except Exception as e:
                    print(f"测试错误: {e}")
                
                # 测试 ComfyUI 配置工具
                print("\n⚙️ 测试 get_comfyui_config 工具...")
                try:
                    result = await session.call_tool(
                        name="get_comfyui_config",
                        arguments={}
                    )
                    config_data = json.loads(result.content[0].text)
                    print(f"ComfyUI 服务器: {config_data.get('server_url', 'N/A')}")
                    print(f"连接状态: {'✓ 可访问' if config_data.get('connectivity', {}).get('server_accessible') else '✗ 不可访问'}")
                    print(f"回退模式: {'启用' if config_data.get('config', {}).get('enable_fallback') else '禁用'}")
                except Exception as e:
                    print(f"配置测试错误: {e}")
                
                # 测试图像生成工具
                print("\n🎨 测试 generate_image_with_context 工具...")
                try:
                    result = await session.call_tool(
                        name="generate_image_with_context",
                        arguments={
                            "prompt": "一只可爱的小猫坐在花园里",
                            "workflow_type": "text_to_image",
                            "width": 512,
                            "height": 512,
                            "steps": 20
                        }
                    )
                    response_data = json.loads(result.content[0].text)
                    
                    if 'error' in response_data:
                        print(f"生成错误: {response_data['error']}")
                    elif 'type' in response_data and response_data['type'] == 'image':
                        metadata = response_data.get('metadata', {})
                        print(f"✓ 图像生成成功")
                        print(f"  尺寸: {metadata.get('dimensions', 'N/A')}")
                        print(f"  生成时间: {metadata.get('generation_time', 'N/A')}秒")
                        print(f"  种子: {metadata.get('seed', 'N/A')}")
                        if 'note' in metadata:
                            print(f"  注意: {metadata['note']}")
                    else:
                        print(f"意外响应格式: {response_data}")
                except Exception as e:
                    print(f"图像生成测试错误: {e}")
                
                # 测试视频生成工具
                print("\n🎬 测试 generate_video_with_context 工具...")
                try:
                    result = await session.call_tool(
                        name="generate_video_with_context",
                        arguments={
                            "prompt": "一只小猫在草地上玩耍",
                            "workflow_type": "text_to_video",
                            "steps": 15,
                            "frame_rate": 8
                        }
                    )
                    response_data = json.loads(result.content[0].text)
                    
                    if 'error' in response_data:
                        print(f"生成错误: {response_data['error']}")
                    elif 'type' in response_data and response_data['type'] == 'video':
                        metadata = response_data.get('metadata', {})
                        print(f"✓ 视频生成成功")
                        print(f"  帧率: {metadata.get('frame_rate', 'N/A')} fps")
                        print(f"  生成时间: {metadata.get('generation_time', 'N/A')}秒")
                        print(f"  文件名: {metadata.get('filename', 'N/A')}")
                        if 'note' in metadata:
                            print(f"  注意: {metadata['note']}")
                    else:
                        print(f"意外响应格式: {response_data}")
                except Exception as e:
                    print(f"视频生成测试错误: {e}")
                
                print("\n🎉 MCP server 测试完成！")
                
    except Exception as e:
        print(f"❌ 测试失败: {e}")

# 运行测试
await test_deployed_mcp_server()

## Test Deployed MCP Server

The code cell above will directly test the MCP server deployed to AgentCore Runtime.

## Next Steps

Now that you have successfully deployed the MCP server to AgentCore Runtime, you can:

1. **Add More Tools**: Extend your MCP server to include more tools
2. **Custom Authentication**: Implement custom JWT authorizers
3. **Integration**: Integrate with other AgentCore services
4. **Monitoring**: Set up logging and monitoring
5. **Optimization**: Optimize performance based on usage patterns

## Clean Up Resources (Optional)

If you want to clean up the resources created in this tutorial, run the following cell:

In [ ]:
import boto3

def cleanup_resources():
    """清理创建的 AWS 资源"""
    print("🗑️ 开始清理过程...")
    
    try:
        # 删除 AgentCore Runtime
        print("删除 AgentCore Runtime...")
        agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
        runtime_delete_response = agentcore_control_client.delete_agent_runtime(
            agentRuntimeId=launch_result.agent_id,
        )
        print("✓ AgentCore Runtime 删除已启动")

        # 删除 ECR 仓库
        print("删除 ECR 仓库...")
        ecr_client = boto3.client('ecr', region_name=region)
        ecr_repo_name = launch_result.ecr_uri.split('/')[1]
        ecr_client.delete_repository(
            repositoryName=ecr_repo_name,
            force=True
        )
        print("✓ ECR 仓库已删除")

        # 删除 IAM 角色
        print("删除 IAM 角色策略...")
        iam_client = boto3.client('iam')
        role_name = agentcore_iam_role['Role']['RoleName']
        
        # 删除角色策略
        policies = iam_client.list_role_policies(
            RoleName=role_name,
            MaxItems=100
        )

        for policy_name in policies['PolicyNames']:
            iam_client.delete_role_policy(
                RoleName=role_name,
                PolicyName=policy_name
            )
        
        # 删除角色
        iam_client.delete_role(RoleName=role_name)
        print("✓ IAM 角色已删除")

        # 删除 Parameter Store 参数
        try:
            ssm_client.delete_parameter(Name='/mcp_server_sample/runtime/agent_arn')
            print("✓ Parameter Store 参数已删除")
        except ssm_client.exceptions.ParameterNotFound:
            print("ℹ️ Parameter Store 参数未找到")

        # 删除 Secrets Manager 密钥
        try:
            secrets_client.delete_secret(
                SecretId='mcp_server_sample/cognito/credentials',
                ForceDeleteWithoutRecovery=True
            )
            print("✓ Secrets Manager 密钥已删除")
        except secrets_client.exceptions.ResourceNotFoundException:
            print("ℹ️ Secrets Manager 密钥未找到")

        # 删除 Cognito 用户池
        try:
            cognito_client = boto3.client('cognito-idp')
            cognito_client.delete_user_pool(UserPoolId=cognito_config['user_pool_id'])
            print("✓ Cognito 用户池已删除")
        except Exception as e:
            print(f"ℹ️ 删除 Cognito 用户池时出错: {e}")

        print("\n✅ 清理完成！")
        
    except Exception as e:
        print(f"❌ 清理过程中出错: {e}")
        print("您可能需要手动清理一些资源。")

# 取消注释下面的行来执行清理
# cleanup_resources()

# 🎉 Congratulations!

You have successfully:

✅ **Created an adapted MCP server** containing existing tools  
✅ **Configured authentication** using Amazon Cognito  
✅ **Deployed to AWS** using AgentCore Runtime  
✅ **Remote invocation** with proper authentication  
✅ **Learned MCP concepts** and best practices  

Your MCP server is now running on Amazon Bedrock AgentCore Runtime and ready for production use!

## Summary

In this tutorial, you learned how to:
- Adapt existing Lambda MCP server tools to FastMCP
- Configure stateless HTTP transport for AgentCore compatibility
- Set up JWT authentication using Amazon Cognito
- Deploy and manage MCP server on AWS
- Test deployed MCP server directly

The deployed MCP server can now be integrated into larger AI applications and workflows!

## Tool Functionality Description

Tools included in this example:

1. **search_website**: 使用 SerpAPI 进行网站搜索
2. **count_s3_buckets**: 计算 AWS S3 存储桶数量
3. **generate_image_with_context**: 图像生成工具（支持真实 ComfyUI 集成）
   - 支持文本到图像、图像到图像、修复等工作流
   - 自动检测 ComfyUI 服务器连接状态
   - 智能回退到模拟响应（当 ComfyUI 不可用时）
   - 完整的错误处理和重试机制
4. **get_comfyui_config**: 获取 ComfyUI 配置信息和连接状态
   - 实时连接性检测
   - 配置参数和预设信息
   - 推荐的图像尺寸和工作流设置
5. **generate_video_with_context**: 视频生成工具（支持真实 ComfyUI 集成）
   - 支持文本到视频、图像到视频工作流
   - 自动处理视频编码和格式转换
   - 扩展超时处理（视频生成通常需要更长时间）

### New Features

- **Environment Variable Configuration**: Flexible configuration of ComfyUI server URL and parameters via environment variables
- **Connectivity Detection**: Automatic detection and reporting of ComfyUI server status
- **Direct Error Handling**: Clear error messages when ComfyUI is unavailable
- **Complete Workflow**: Support for complete ComfyUI workflow submission, polling, and result retrieval

These tools demonstrate different types of functionality: API calls, AWS service integration, AI generation tasks, etc., particularly showing how to integrate external AI services in AgentCore Runtime.